# TokenCountingHandler - Demo Usage

This notebook walks through how to use the TokenCountingHandler and how it can be used to track your prompt, completion, and embedding token usage over time.

In [1]:
import tiktoken
from langchain.chat_models import ChatOpenAI

from llama_index import (
    SimpleDirectoryReader, 
    LLMPredictor, 
    VectorStoreIndex, 
    ServiceContext,
)
from llama_index.callbacks import (
    CallbackManager, 
    TokenCountingHandler
)

/home/lmarkewi/miniconda3/envs/llama_index/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup

Here, we setup the callback and the serivce context. We set a global service context so that we don't have to worry about passing it into indexes and queries.

In [2]:
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

llm_predictor = LLMPredictor(
    llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
)

# set the global default!
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, callback_manager=callback_manager).set_global()

## Token Counting

The token counter will track embedding, prompt, and completion token usage. The token counts are __cummulative__ and are only reset when you choose to do so, with `token_counter.reset_counts()`.

### Embedding Token Usage

Now that the service context is setup, let's track our embedding token usage.

In [3]:
documents = SimpleDirectoryReader("../data/paul_graham").load_data()

In [4]:
index = VectorStoreIndex.from_documents(documents)

In [5]:
print(token_counter.total_embedding_token_count)

16830


That looks right! Before we go any further, lets reset the counts

In [6]:
token_counter.reset_counts()

### LLM + Embedding Token Usage

Next, let's test a query and see what the counts look like.

In [7]:
query_engine = index.as_query_engine(similarity_top_k=4)
respons = query_engine.query("What did the author do growing up?")

In [8]:
print('Embedding Tokens: ', token_counter.total_embedding_token_count, '\n',
      'LLM Prompt Tokens: ', token_counter.prompt_llm_token_count, '\n',
      'LLM Completion Tokens: ', token_counter.completion_llm_token_count, '\n',
      'Total LLM Token Count: ', token_counter.total_llm_token_count, '\n')

Embedding Tokens:  8 
 LLM Prompt Tokens:  3969 
 LLM Completion Tokens:  124 
 Total LLM Token Count:  4093 



So, LlamaIndex used 8 token to embed the query text, sent 3987 prompt tokens to the LLM, and received 149 completion tokens from the LLM.

## Advanced Usage

The token counter tracks each token usage event in an object called a `TokenCountingEvent`. This object has the following attributes:

- prompt -> The prompt string sent to the LLM or Embedding model
- prompt_token_count -> The token count of the LLM prompt
- completion -> The string completion received from the LLM (not used for embeddings)
- completion_token_count -> The token count of the LLM completion (not used for embeddings)
- total_token_count -> The total prompt + completion tokens for the event
- event_id -> A string ID for the event, which aligns with other callback handlers

These events are tracked on the token counter in two lists:

- llm_token_counts
- embedding_token_counts

Let's explore what these look like!

In [9]:
print('Num LLM token count events: ', len(token_counter.llm_token_counts))
print('Num Embedding token count events: ', len(token_counter.embedding_token_counts))

Num LLM token count events:  2
Num Embedding token count events:  1


This makes sense! The previous query embedded the query text, and then made 2 LLM calls (since the top k was 4, and the default chunk size is 1024, two seperate calls need to be made so the LLM can read all the retrieved text).

Next, let's quickly see what these events look like for a single event.

In [10]:
print('prompt: ', token_counter.llm_token_counts[0].prompt[:100], '...\n')
print('prompt token count: ', token_counter.llm_token_counts[0].prompt_token_count, '\n')

print('completion: ', token_counter.llm_token_counts[0].completion[:100], '...\n')
print('completion token count: ', token_counter.llm_token_counts[0].completion_token_count, '\n')

print('total token count', token_counter.llm_token_counts[0].total_token_count)

prompt:  Context information is below. 
---------------------
I could write essays again, I wrote a bunch abo ...

prompt token count:  3261 

completion:  The author wrote short stories and programmed on an IBM 1401 in 9th grade, and later on a TRS-80 mic ...

completion token count:  32 

total token count 3293
